In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import bisect
import warnings

from torch._utils import _accumulate
from torch import randperm
import torch
import torch.utils.data

In [16]:
#IMPORT DATA
data = pd.read_csv("hmnist_64_64_L.csv")
Y = data["label"]
data.drop(["label"],axis=1, inplace=True)
X = data
# SPLIT DATA INTO TRAIN, TEST, AND VALIDATION SET
#train_data, val_data, test_data = torch.utils.data.random_split(data_, (3500,750,750))
#print(type(test_data))
#train_labels = train_data["label"]
#val_label = val_label["label"]
#test_data = test_data["label"]
#train_data.drop(["label"],axis=1, inplace=True)
#val_data.drop(["label"],axis=1, inplace=True)
#test_data.drop(["label"],axis=1, inplace=True)
#train_X = train_data
#val_X = val_data
#test_X = test_data

In [4]:
#NORMALISING THE DATASET
X = X/ 255

In [5]:
print(X.shape)

(5000, 4096)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import torch.optim as optim

In [7]:
X=np.asarray(X)
#X = X.values.reshape(-1,64,64,1)
Y = Y.values
#input_shape = (64,64,1)
X = Tensor(X)
X = X.view(5000,1,64,64)
#Y = Tensor(Y)
X = np.array(X)

print(X.shape)


(5000, 1, 64, 64)


In [8]:
class ConvNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(1, 128, kernel_size = 5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
            
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.hidden = nn.Linear(6*6*64, 256)
        self.drop = nn.Dropout(0.25)
        self.func = nn.ReLU()
        self.out1 = nn.Linear(256,64)
        self.out2 = nn.Linear(64,32)
        self.out3 = nn.Linear(32,9)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.drop(x)
        x = self.conv2(x)
        x = self.drop(x)
        x = self.conv3(x)
        x = self.drop(x)
        x = x.reshape(x.size(0), -1)
        x = self.func(self.hidden(x))
        x = self.func(self.out1(x))
        x = self.func(self.out2(x))
        x = self.out3(x)
        return x

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [10]:
print("x_train.shape: ",x_train.shape)
print("x_val.shape: ",x_val.shape)
print("y_train.shape: ",y_train.shape)
print("y_val.shape: ",y_val.shape)

x_train.shape:  (4000, 1, 64, 64)
x_val.shape:  (1000, 1, 64, 64)
y_train.shape:  (4000,)
y_val.shape:  (1000,)


In [11]:
# utility for getting prediction accuracy
def get_correct_and_accuracy(y_pred, y):
    # y_pred is the nxC prediction scores
    # give the number of correct and the accuracy
    n = y.shape[0]
    # find the prediction class label
    _ ,pred_class = y_pred.max(dim=1)
    correct = (pred_class == y).sum().item()
    return correct ,correct/n

In [12]:
# init network
conv_net = ConvNet(4096, 9)
print('model structure: ', conv_net)
# init optimizer
optimizer = torch.optim.Adam(conv_net.parameters(),lr=1e-3)
# set loss function
criterion = nn.CrossEntropyLoss()

# prepare for mini-batch stochastic gradient descent
n_iteration = 40
batch_size = 256
n_data = X.shape[0]
# print(n_data)
n_batch = int(np.ceil(n_data/batch_size))

# convert X_train and X_val to tensor and flatten them
X_train_tensor = Tensor(x_train)
#X_test_tensor = Tensor(x_test)
X_val_tensor = Tensor(x_val)

# convert training label to tensor and to type long
y_train_tensor = Tensor(y_train).long()
#y_test_tensor = Tensor(y_test).long()
y_val_tensor = Tensor(y_val).long()

print('X train tensor shape:', X_train_tensor.shape)

model structure:  ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (hidden): Linear(in_features=2304, out_features=256, bias=True)
  (drop): Dropout(p=0.25)
  (func): ReLU()
  (out1): Linear(in_features=256, out_features=64, bias=True)
  (out2): Linear(in_features=64, out_features=32, bias=True)
  (out3): Linear(in_features=32, out_features=9, bias=True)
)
X train tensor shape: torch.Size([4000, 1, 64, 64])


In [13]:
n_batch = 15

In [14]:

## start 
train_loss_list = np.zeros(n_iteration)
train_accu_list = np.zeros(n_iteration)
val_loss_list = np.zeros(n_iteration)
val_accu_list = np.zeros(n_iteration)
for i in range(n_iteration):
    # first get a minibatch of data
    train_loss = 0
    train_accu = 0
    for j in range(n_batch):
        batch_start_index = j*batch_size
        # get data batch from the normalized data
        X_batch = X_train_tensor[batch_start_index:batch_start_index+batch_size]
        # get ground truth label y
        y_batch = y_train_tensor[batch_start_index:batch_start_index+batch_size]
        
        y_pred = conv_net(X_batch)
        loss = criterion(y_pred, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        batch_correct, batch_accuracy = get_correct_and_accuracy(y_pred, y_batch)
        train_accu += batch_accuracy
        
    ave_train_loss = train_loss/(j)
    ave_train_accu = train_accu/(j)
    
    y_pred_validation = conv_net(X_val_tensor)
    val_loss = criterion(y_pred_validation,y_val_tensor)
    val_correct, val_accu = get_correct_and_accuracy(y_pred_validation, y_val_tensor)
    
    
    print("Iter %d ,Train loss: %.3f, Train acc: %.3f, Val loss: %.3f, Val acc: %.3f" 
          %(i ,ave_train_loss, train_accu, val_loss, val_accu)) 
    
    ## add to the logs so that we can use them later for plotting
    train_loss_list[i] = ave_train_loss
    train_accu_list[i] = ave_train_accu
    val_loss_list[i] = val_loss
    val_accu_list[i] = val_accu

Iter 0 ,Train loss: 2.193, Train acc: 1.719, Val loss: 1.902, Val acc: 0.126
Iter 1 ,Train loss: 1.896, Train acc: 2.996, Val loss: 1.555, Val acc: 0.331
Iter 2 ,Train loss: 1.500, Train acc: 6.004, Val loss: 1.336, Val acc: 0.379
Iter 3 ,Train loss: 1.302, Train acc: 6.961, Val loss: 1.173, Val acc: 0.508
Iter 4 ,Train loss: 1.210, Train acc: 7.578, Val loss: 1.131, Val acc: 0.548
Iter 5 ,Train loss: 1.178, Train acc: 7.832, Val loss: 1.080, Val acc: 0.570
Iter 6 ,Train loss: 1.141, Train acc: 8.320, Val loss: 1.099, Val acc: 0.552


KeyboardInterrupt: 

In [ ]:
## plot training loss versus validation loss
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
x_axis = np.arange(n_iteration)
plt.plot(x_axis, train_loss_list, label='train loss')
plt.plot(x_axis, val_loss_list, label='val loss')
plt.legend()
plt.show()

In [ ]:
## plot training accuracy versus validation accuracy
plt.plot(x_axis, train_accu_list, label='train acc')
plt.plot(x_axis, val_accu_list, label='val acc')
plt.legend()
plt.show()

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(1, 128, kernel_size = 5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
            
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.hidden = nn.Linear(6*6*64, 256)
        self.drop = nn.Dropout(0.25)
        self.func = nn.ReLU()
        self.out1 = nn.Linear(256,64)
        self.out2 = nn.Linear(64,32)
        self.out3 = nn.Linear(32,9)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.drop(x)
        x = self.conv2(x)
        x = self.drop(x)
        x = self.conv3(x)
        x = self.drop(x)
        x = x.reshape(x.size(0), -1)
        x = self.func(self.hidden(x))
        x = self.drop(x)
        x = self.func(self.out1(x))
        x = self.func(self.out2(x))
        x = self.out3(x)
        return x
        

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 6, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=6, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(16384, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.reshape(x.size(0), -1)
        x = self.drop_out(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x